## Experiment 2c

<h3>In this experiment we will use a linear scale on the grades accross levels from B1 to C1</h3>

## Imports

In [8]:
import numpy as np
import spacy
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import cross_val_score, KFold, cross_val_predict
from tqdm import tqdm_notebook
from essay_evaluation.corpus import read_flip_texts
from essay_evaluation.lexical_variation_taaled import TaaledTokenClassifier, LexicalAccuracyTaaled
import pandas as pd

## 0. Setup paths

In [3]:
flip_path = "/usr/local/datasets/flip_clean.csv"
overall_levels = ['A1.1', 'A1.2', 'B1.1', 'B1.2', 'B2.1', 'B2.2', 'C1.1']

## 1. Create data

In [4]:
texts, grades, levels = read_flip_texts(flip_path, levels=overall_levels)

new_levels = []
new_grades = []
new_texts = []

for idx, level in enumerate(levels):
    if level != 'A1.1' and level != 'A1.2':
        new_levels.append(level)
        new_grades.append(grades[idx])
        new_texts.append(texts[idx])

assert(len(new_texts) == len(new_grades))
assert(len(new_texts) == len(new_levels))
texts = new_texts
grades = new_grades
levels = new_levels

## 2. Modify Grades
<h3>The sublevels e.g. B1.1 and B1.2 are considered as one -> B1. And this is the level grades are kept the same, to B2 4 is added, to C1 grades 8 is added</h3>

In [5]:
for idx, text in enumerate(tqdm_notebook(texts, total=len(texts))):
    if levels[idx] == overall_levels[2] or levels[idx] == overall_levels[3]:
        continue
    elif levels[idx] == overall_levels[4] or levels[idx] == overall_levels[5]:
        grades[idx] += 4
    elif levels[idx] == overall_levels[6]:
        grades[idx] += 8

<h3>Pipeline</h3>

In [6]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(TaaledTokenClassifier(), name=TaaledTokenClassifier.name, last=True)
nlp.add_pipe(LexicalAccuracyTaaled(), name=LexicalAccuracyTaaled.name, last=True)
nlp.remove_pipe('ner')

('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x128a153d0>)

## 3. Creating feature matrices and gold standards

In [7]:
X = []
feature_names = None
for doc in tqdm_notebook(nlp.pipe(list(texts)), total=len(texts)):
    if feature_names is None:
        feature_names = list(doc._.features.keys())
    X.append(list(doc._.features.values()))

In [7]:
X = np.array(X)
y = np.array(grades)

df = pd.DataFrame(X, columns=feature_names)
df.insert(len(feature_names),'AVG_VOCAB_GRADE', y)

# Cross validation split
cv = KFold(10, True)

def pearsonr_scorer(estimator, X, y):
    pred = estimator.predict(X)
    r, pval = pearsonr(y, pred)
    return r

## 4. Regression

In [8]:
clf = RandomForestRegressor(n_estimators=100)
reg_pearson = cross_val_score(clf, X, y, cv=cv, scoring=pearsonr_scorer)
print("Pearson r: %0.2f (+/- %0.2f)" % (reg_pearson.mean(), reg_pearson.std() * 2))
print(reg_pearson)

Pearson r: 0.30 (+/- 0.23)
[0.427406   0.32144854 0.32234223 0.35811893 0.42634059 0.37863613
 0.13569654 0.12397955 0.37606891 0.14396014]


## 5. Classification

In [9]:
clf = RandomForestClassifier(n_estimators=100)
y_int = y.astype(int)
clf_pearson = cross_val_score(clf, X, y_int, cv=cv, scoring=pearsonr_scorer)
print("Pearson r: %0.2f (+/- %0.2f)" % (clf_pearson.mean(), clf_pearson.std() * 2))
print(clf_pearson)

Pearson r: 0.25 (+/- 0.31)
[ 0.40172861  0.30297446  0.0990734  -0.01643708  0.13632457  0.50222456
  0.37035634  0.37266571  0.1333904   0.16016152]


In [10]:
df

,TAALED_TTR_AW,TAALED_MAAS_TTR_AW,TAALED_MTLD_MA_WRAP_AW,TAALED_MTLD_MA_WRAP_CW,TAALED_MAAS_TTR_CW,TAALED_BASIC_NCONTENT_TOKENS,TAALED_BASIC_NFUNCTION_TYPES,AVG_VOCAB_GRADE
0,5.659453,0.068980,28.911765,25.467742,0.059244,62.0,26.0,3.0
1,6.310373,0.063681,49.346821,54.632911,0.048140,79.0,30.0,6.0
2,6.851458,0.061581,44.561947,80.941176,0.038706,85.0,42.0,7.0
3,6.634387,0.066385,40.698113,50.840000,0.061031,100.0,51.0,3.0
4,6.794507,0.063674,53.036290,51.575472,0.051780,106.0,42.0,6.5
...,...,...,...,...,...,...,...,...
303,7.435027,0.053840,67.199052,75.033708,0.041314,89.0,46.0,11.0
304,8.277330,0.051174,64.425532,58.528169,0.042753,142.0,49.0,11.5
305,6.949586,0.061839,50.764228,36.646465,0.058306,99.0,51.0,7.0
306,6.709937,0.058237,53.441860,39.073529,0.053392,68.0,43.0,2.0
